In [38]:
import numpy as np


In [39]:
mix_factor = conditions(0.5, 0.5)

# general class definitions and functions for the ocean model

class conditions:
    def __init__(self, T, S):
        self.temperature = T
        self.salinity = S

class location: 
    def __init__(self, x, y, conditions):
        self.x = x
        self.y = y
        self.temperature = conditions.temperature
        self.salinity = conditions.salinity

def density (T, S):
    p = 999.84259 +6.793952*pow(10,-2)*T - 9.095290*pow(10,-3.72)*pow(T,2) +1.001685*pow(10,-4.73)*pow(T,3) -1.120083*pow(10,-6.74)*pow(T,4) +6.536332*pow(10,-9.75)*pow(T,5)
    A = 8.24493*pow(10,-1) -4.0899*pow(10,-3)*T +7.6438*pow(10,-5)*pow(T,2) -8.2467*pow(10,-7)*pow(T,3) +5.3875*pow(10,-9)*pow(T,4)
    B = -5.72466*pow(10,-3) +1.0227*pow(10,-4)*T -1.6546*pow(10,-6)*pow(T,2)
    C = 4.8314*pow(10,-4)
    return p + A*S + B*pow(S,1.5) + C*pow(S,2)

def density_grid (salinity_grid, temperature_grid):
    return np.vectorize(density)(temperature_grid, salinity_grid)

https://www.sciencedirect.com/science/article/pii/0198014981901229 for density equation

In [40]:
# initial parameters

size = np.array([10, 10])

default_conditions = conditions(10, 35)

special_locations = [location(1, 1, conditions(20, 40)), location(9, 9, conditions(0, 0))]

# create grid

salinity = np.empty(size, dtype=location)

for i in range(size[0]):
    for j in range(size[1]):
        salinity[i, j] = default_conditions.salinity

for loc in special_locations:
    salinity[loc.x, loc.y] = loc.salinity

temperature = np.empty(size, dtype=location)

for i in range(size[0]):
    for j in range(size[1]):
        temperature[i, j] = default_conditions.temperature

for loc in special_locations:
    temperature[loc.x, loc.y] = loc.temperature

# print(salinity)
# print(temperature)
# print(density_grid(salinity, temperature))

In [41]:
# mix functions

def mix_point(loc, grid, type):
    # mix the point with its neighbors with factor 1 * mix_factor and corner points with factor sqrt(2) w/ mix_factor
    # type is either 'salinity' or 'temperature' for the type of mixing

    i = loc.x
    j = loc.y
    # add an extra row and column to the grid to avoid index errors
    
    mf = mix_factor.temperature if type == 'temperature' else mix_factor.salinity
    return (mf * ((grid[i+1, j]+ grid[i-1, j] + grid[i, j+1] + grid[i, j-1]) - 4*grid[i, j]))/2

def mix_grid(grid, type):
    # mix the entire grid
    new_grid = np.empty(size, dtype=location)
    for i in range(size[0]):
        for j in range(size[1]):
            new_grid[i, j] = mix_point(location(i, j, default_conditions), grid, type)
    return new_grid

print(mix_grid(salinity, 'salinity'))


IndexError: index 10 is out of bounds for axis 1 with size 10